## Criar o Assistente

In [2]:

import openai
import os
from dotenv import find_dotenv, load_dotenv

# load the api key
_ = load_dotenv(find_dotenv())
client = openai.Client(api_key=os.getenv("API_KEY"))

### criando o assistente

In [3]:
assistant = client.beta.assistants.create(
    name="Tutor de matemática",
    instructions="Você é um tutor de matemática e deve ajudar os alunos a resolver problemas de matemática. Escreva e execute o código para resolver os problemas.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

### Criando uma Thread

In [5]:
thread = client.beta.threads.create()

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Qual a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes aproximadamente?"
)

### Rodando a threead

In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="O nome do usuário é Wellington e ele é um usuário Premium."
)

### Aguardart a thread rodar

In [8]:
import time
# roda várias vezes para identificar o status
while run.status in  ["queued", "in_progress", "cancelling" ]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run.status)

in_progress
in_progress
in_progress
in_progress
in_progress
completed


### Quando terminar de roddar

In [9]:
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages)
else:
    print(run.error)

SyncCursorPage[Message](data=[Message(id='msg_cFLS9ksrc1aH7j2M7WPZ1bvL', assistant_id='asst_rHJyuZQBplQ2Qz17nCiUlfv4', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, aproximadamente 1.26%.'), type='text')], created_at=1741648757, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_9evRmcoNNrSDnSD2MXUrwQsD', status=None, thread_id='thread_GEnGqKFXau2VWx6Ex06SDfV3'), Message(id='msg_k3eGTxG2PIWgnR4M49JZcpFi', assistant_id='asst_rHJyuZQBplQ2Qz17nCiUlfv4', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fórmula da distribuição binomial é dada por:\n\n\\[ P(X = k) = 

In [10]:
print(messages.data[0].content[0].text.value)

A probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, aproximadamente 1.26%.


### Analisando os passos do modelo

In [11]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [12]:
run_steps.data[0].step_details

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_cFLS9ksrc1aH7j2M7WPZ1bvL'), type='message_creation')

In [13]:
for step in run_steps.data[::-1]:
    print("#step:", step.step_details.type)
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("> tool_call type:", tool_call.type)
            print("> input:")
            print(tool_call.code_interpreter.input)
            print("> outputs:", tool_call.code_interpreter.outputs)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id, message_id=step.step_details.message_creation.message_id
        )
        print("--- message_text:", message.content[0].text.value)


#step: message_creation
--- message_text: Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fórmula da distribuição binomial é dada por:

\[ P(X = k) = \binom{n}{k} \times p^k \times (1-p)^{n-k} \]

onde:
- \( P(X = k) \) é a probabilidade de obter exatamente \(k\) sucessos
- \( n \) é o número total de tentativas
- \( k \) é o número exato de sucessos desejados
- \( p \) é a probabilidade de sucesso em uma única tentativa

No nosso caso, \( n = 1000 \) (número de lançamentos), \( k = 150 \) (número de vezes que queremos obter o número 6), e \( p = \frac{1}{6} \) (probabilidade de obter o número 6 em um dado lançamento).

Vamos calcular a probabilidade utilizando esses valores.
#step: tool_calls
> tool_call type: code_interpreter
> input:
from math import comb

# Definindo os parâmetros
n = 1000
k = 150
p = 1/6

# Calculando a probabilidade
probability = comb(n, k) * (p**k) * ((1-p)**(n-k